In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/opt/continuum/anaconda/bin/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.7.0_67-cloudera/jre"
os.environ["SPARK_HOME"] = "/opt/cloudera/parcels/CDH/lib/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.9-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
conda install -c conda-forge/label/cf201901 pyspark

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install -c cyclus java-jdk

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf())

In [131]:
from collections import defaultdict,OrderedDict
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel
import re

In [132]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark = SparkSession \
    .builder \
    .appName("DataFrame Intro") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
sc1 = spark.sparkContext

In [133]:
#raw = sc.textFile('Documents/BigDataTech/trump.txt')
df = spark.read.format("csv").option("header", "false").load("Documents/BigDataTech/trump.csv")
df.take(1)

[Row(_c0='Twitter for iPhone', _c1='Trump flags being waived at the Bibi @Netanyahu VICTORY celebration last night! https://t.co/SX8RVAALYW', _c2='04-10-2019 14:48:02', _c3='3985', _c4='14533', _c5='false', _c6='1115989820935819264')]

In [134]:
tweets = df[['_c1']]
tweets.take(1)

[Row(_c1='Trump flags being waived at the Bibi @Netanyahu VICTORY celebration last night! https://t.co/SX8RVAALYW')]

In [135]:
tmp = tweets.rdd.flatMap(list)
tmp.take(5)

['Trump flags being waived at the Bibi @Netanyahu VICTORY celebration last night! https://t.co/SX8RVAALYW',
 'Everybody is now acknowledging that right from the time I announced my run for President I was 100% correct on the Border. Remember the heat I took? Democrats should now get rid of the loopholes. The Border is being fixed. Mexico will not let people through!',
 'The Democrats must end the loopholes on immigration. So easy to solve!',
 '“The underlying issue remains the same without a single American being indicted for Collusion with Russia &amp; that is a stunning end considering we were led to believe (Fake News) that something much more dramatic would happen. Why did people fall for this?” @TuckerCarlson Molly H',
 'MAKE AMERICA GREAT AGAIN! https://t.co/diXWQHuyGj']

In [136]:
dtrump = tmp.map(lambda x: re.sub('[,/_\."“;!*&()?”]','',x))
dtrump.take(5)

['Trump flags being waived at the Bibi @Netanyahu VICTORY celebration last night https:tcoSX8RVAALYW',
 'Everybody is now acknowledging that right from the time I announced my run for President I was 100% correct on the Border Remember the heat I took Democrats should now get rid of the loopholes The Border is being fixed Mexico will not let people through',
 'The Democrats must end the loopholes on immigration So easy to solve',
 'The underlying issue remains the same without a single American being indicted for Collusion with Russia amp that is a stunning end considering we were led to believe Fake News that something much more dramatic would happen Why did people fall for this @TuckerCarlson Molly H',
 'MAKE AMERICA GREAT AGAIN https:tcodiXWQHuyGj']

In [137]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/janaywilliams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [138]:
something = dtrump.flatMap(lambda x: x.split(','))
words = something.flatMap(lambda x: x.split(' ')).filter(lambda x: len(x)>1).map(lambda x: x.lower())

In [139]:
stop_words = set(stopwords.words('english'))
finals = words.filter(lambda x: x not in stop_words).map(lambda x: (x,1))

In [140]:
dt = dtrump.flatMap(lambda x: x.split(' ')).map(lambda x: x.lower()).filter(lambda x: x not in stop_words).filter(lambda x: len(x)>1).distinct().zipWithIndex()

In [141]:
mydictionary = dt.collectAsMap()

In [142]:
new_dct = dict()
for k,v in mydictionary.items(): new_dct[v]=k

In [143]:
sorted_keys, sorted_values = zip(*sorted(mydictionary.items()))

In [144]:
def to_sparseVector(doc): 
    return [id, Vectors.sparse(len(new_dct), 
                               sorted_keys, 
                               sorted_values)]

In [145]:
corpus = dt.map(to_sparseVector)

In [146]:
num_topics = 5
num_words_in_topic = 10

lda_model = LDA.train(corpus, k=num_topics, maxIterations=50)
topics = lda_model.describeTopics(maxTermsPerTopic=num_words_in_topic)

for idx in range(num_topics):
    print("Topic #{0}".format(idx))
    for i in range(num_words_in_topic):
        print("  {0}\t{1}".format(new_dct[topics[idx][0][i]], topics[idx][1][i]))

    print("")

print("Vocabulary size = {0}".format(len(mydictionary)))

Py4JJavaError: An error occurred while calling o1769.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 96.0 failed 1 times, most recent failure: Lost task 0.0 in stage 96.0 (TID 104, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-inpu